# Ratebeer Text Feature Engineering


In [2]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import fasttext
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from surprise import SVD
from surprise import Dataset
from surprise import evaluate, print_perf, Reader, dump
from sklearn.model_selection import train_test_split
import os

In [2]:
df = pd.read_pickle('./train_set.pkl')

In [3]:
df.head()

,beerID,beer_name,abv,abv_listed,brewerID,beer_style,reviewer_username,review_appearance,review_aroma,review_palate,...,vader_pos,time_from_beginning,weekend_review,time_from_beginning_lookup,weekday_central_lookup,month_lookup,distance_from_july_lookup,vader_neg_lookup,vader_neu_lookup,vader_pos_lookup
367332,5868,St Peters Organic Ale,4.5,True,974,Bitter,abemorsten,3,6,4,...,0.179,300412800,True,"[5868, abemorsten, 300412800]","[5868, abemorsten, Sunday]","[5868, abemorsten, 10]","[5868, abemorsten, 3]","[5868, abemorsten, 0.064]","[5868, abemorsten, 0.757]","[5868, abemorsten, 0.179]"
1841875,493,Busch Light,4.1,True,84,Pale Lager,beerdrinker101,1,1,1,...,0.100,186105600,True,"[493, beerdrinker101, 186105600]","[493, beerdrinker101, Sunday]","[493, beerdrinker101, 03]","[493, beerdrinker101, 4]","[493, beerdrinker101, 0.0]","[493, beerdrinker101, 0.9]","[493, beerdrinker101, 0.1]"
1420827,85344,Auburn Alehouse Red 45,6.0,True,8572,Irish Ale,bb,4,7,3,...,0.000,250473600,False,"[85344, bb, 250473600]","[85344, bb, Wednesday]","[85344, bb, 03]","[85344, bb, 4]","[85344, bb, 0.0]","[85344, bb, 1.0]","[85344, bb, 0.0]"
1709329,1763,Greene King Abbot Ale &#40;Filtered&#41;,5.0,True,143,Premium Bitter/ESB,jazz88,4,5,3,...,0.107,94867200,False,"[1763, jazz88, 94867200]","[1763, jazz88, Monday]","[1763, jazz88, 04]","[1763, jazz88, 3]","[1763, jazz88, 0.128]","[1763, jazz88, 0.764]","[1763, jazz88, 0.107]"
327581,2396,Atlantic Coal Porter,5.8,True,253,Porter,zach8270,3,6,4,...,0.000,141004800,False,"[2396, zach8270, 141004800]","[2396, zach8270, Wednesday]","[2396, zach8270, 09]","[2396, zach8270, 2]","[2396, zach8270, 0.062]","[2396, zach8270, 0.938]","[2396, zach8270, 0.0]"


In [4]:
stopWords = set(stopwords.words('english'))

In [7]:
df['text_filtered'] = df['review_text']\
.apply( lambda x: " ".join([i for i in x.lower().split() if i not in stopWords]) )

In [8]:
# now remove punctuation
df["text_simple"] = df['text_filtered'].str.replace('[^\w\s]','')

In [3]:

#sentiment_scorer = SentimentIntensityAnalyzer()
#df['sentiment_vader'] = df['review_text'].apply(lambda x: sentiment_scorer.polarity_scores(x))
#df1, df2 = np.array_split(df, 2)
#df1.to_pickle(path='./post_vader1.pkl')
#df2.to_pickle(path='./post_vader2.pkl')

df1 = pd.read_pickle('post_vader1.pkl')
df2 = pd.read_pickle('post_vader2.pkl')
df=pd.concat([df1,df2])

In [4]:
df.columns.values

array(['beerID', 'beer_name', 'abv', 'abv_listed', 'brewerID',
       'beer_style', 'reviewer_username', 'review_appearance',
       'review_aroma', 'review_palate', 'review_taste', 'review_overall',
       'review_text', 'review_unix_time', 'weekday_central', 'day_of_year',
       'month', 'distance_from_july', 'text_filtered', 'text_simple',
       'sentiment_vader'], dtype=object)

In [5]:
df.head()

,beerID,beer_name,abv,abv_listed,brewerID,beer_style,reviewer_username,review_appearance,review_aroma,review_palate,...,review_overall,review_text,review_unix_time,weekday_central,day_of_year,month,distance_from_july,text_filtered,text_simple,sentiment_vader
2291588,121,Stone City Hefeweizen,6.0,False,23,German Hefeweizen,lazarus99,5,8,4,...,20,I love this kind of beer. It seems to be the ...,955497600,Tuesday,102,04,3,love kind beer. seems thing apart corona type ...,love kind beer seems thing apart corona type b...,"{'neu': 0.819, 'compound': 0.6369, 'pos': 0.18..."
1651460,132,Newcastle Brown Ale,4.7,True,751,Brown Ale,billb,5,6,5,...,17,This has to be one of my favorite drinking be...,955497600,Tuesday,102,04,3,"one favorite drinking beers. light, dark. good...",one favorite drinking beers light dark good cl...,"{'neu': 0.798, 'compound': 0.8225, 'pos': 0.20..."
2146614,286,Worthington White Shield,5.6,True,12871,Premium Bitter/ESB,wade,3,10,4,...,18,Excellent ale with a strong aroma and initial...,955843200,Saturday,106,04,3,excellent ale strong aroma initial flavor. aft...,excellent ale strong aroma initial flavor afte...,"{'neu': 0.568, 'compound': 0.81, 'pos': 0.432,..."
2182000,544,Saranac Pale Ale,5.5,True,92,English Pale Ale,billb,4,7,5,...,17,Pretty tasty for a pale ale. This is one of ...,956448000,Saturday,113,04,3,pretty tasty pale ale. one faves saranac. nice...,pretty tasty pale ale one faves saranac nice h...,"{'neu': 0.691, 'compound': 0.8807, 'pos': 0.30..."
2171356,547,Saranac Black and Tan,5.1,True,92,Stout,billb,3,7,2,...,10,I a big fan of Black and Tans but this one is...,956448000,Saturday,113,04,3,big fan black tans one rather disappointing. a...,big fan black tans one rather disappointing ar...,"{'neu': 0.701, 'compound': 0.0516, 'pos': 0.16..."


In [6]:
# split vader sentiment scores into seperate columns
df['vader_neg']=df['sentiment_vader'].apply(lambda x: float(x['neg']))
df['vader_neu']=df['sentiment_vader'].apply(lambda x: float(x['neu']))
df['vader_pos']=df['sentiment_vader'].apply(lambda x: float(x['pos']))

Create a feature listing number of seconds since first review on record. This might be easier to train on than the unix time value.

In [7]:
df['review_unix_time'].min()

955497600

In [8]:
df['time_from_beginning']=df['review_unix_time']-955497600

In [9]:
# create boolean feature saying if review is on weekend
df['weekend_review']=df['weekday_central'].apply(lambda x: True if x in ['Friday', 'Saturday', 'Sunday'] else False)

In [10]:
# create this list of lists so we can aggregate into a dict later 
# This approach is stupid! Ended up running out of memory or it became too computationally expensive
"""df['text_lookup'] = df[['beerID','reviewer_username', 'text_simple']].values.tolist()
df['abv_lookup']=df[['beerID', 'reviewer_username', 'abv']].values.tolist()
df['brewerID_lookup']=df[['beerID','reviewer_username','brewerID']].values.tolist()
df['beerStyle_lookup']=df[['beerID','reviewer_username','beer_style']].values.tolist()
df['review_appearance_lookup']=df[['beerID','reviewer_username','review_appearance']].values.tolist()
df['review_aroma_lookup']=df[['beerID', 'reviewer_username', 'review_aroma']].values.tolist()
df['review_palate_lookup']=df[['beerID', 'reviewer_username', 'review_palate']].values.tolist()
df['review_taste_lookup']=df[['beerID', 'reviewer_username', 'review_taste']].values.tolist()
df['review_overall_lookup']=df[['beerID', 'reviewer_username', 'review_overall']].values.tolist()
df['time_from_beginning_lookup']=df[['beerID', 'reviewer_username', 'time_from_beginning']].values.tolist()
df['weekday_central_lookup']=df[['beerID', 'reviewer_username', 'weekday_central']].values.tolist()
df['month_lookup']=df[['beerID', 'reviewer_username', 'month']].values.tolist()
df['distance_from_july_lookup']=df[['beerID', 'reviewer_username', 'distance_from_july']].values.tolist()
df['vader_neg_lookup']=df[['beerID', 'reviewer_username', 'vader_neg']].values.tolist()
df['vader_neu_lookup']=df[['beerID', 'reviewer_username', 'vader_neu']].values.tolist()
df['vader_pos_lookup']=df[['beerID', 'reviewer_username', 'vader_pos']].values.tolist()"""


"df['text_lookup'] = df[['beerID','reviewer_username', 'text_simple']].values.tolist()\ndf['abv_lookup']=df[['beerID', 'reviewer_username', 'abv']].values.tolist()\ndf['brewerID_lookup']=df[['beerID','reviewer_username','brewerID']].values.tolist()\ndf['beerStyle_lookup']=df[['beerID','reviewer_username','beer_style']].values.tolist()\ndf['review_appearance_lookup']=df[['beerID','reviewer_username','review_appearance']].values.tolist()\ndf['review_aroma_lookup']=df[['beerID', 'reviewer_username', 'review_aroma']].values.tolist()\ndf['review_palate_lookup']=df[['beerID', 'reviewer_username', 'review_palate']].values.tolist()\ndf['review_taste_lookup']=df[['beerID', 'reviewer_username', 'review_taste']].values.tolist()\ndf['review_overall_lookup']=df[['beerID', 'reviewer_username', 'review_overall']].values.tolist()\ndf['time_from_beginning_lookup']=df[['beerID', 'reviewer_username', 'time_from_beginning']].values.tolist()\ndf['weekday_central_lookup']=df[['beerID', 'reviewer_username', 

In [11]:
df.columns.values

array(['beerID', 'beer_name', 'abv', 'abv_listed', 'brewerID',
       'beer_style', 'reviewer_username', 'review_appearance',
       'review_aroma', 'review_palate', 'review_taste', 'review_overall',
       'review_text', 'review_unix_time', 'weekday_central', 'day_of_year',
       'month', 'distance_from_july', 'text_filtered', 'text_simple',
       'sentiment_vader', 'vader_neg', 'vader_neu', 'vader_pos',
       'time_from_beginning', 'weekend_review'], dtype=object)

## Aggregate features

### Aggregate by User

In [12]:
# below old version of aggregation before I realised I would need to keep tab of which review corresponds to which 
# value so I could adjust the aggregate values by removing the values corresponding to a specific item/reviewer
# when we join later
"""aggregation={
    'beerID' : {
        'beerID_count' : 'count'
    },
    'abv' : {
        'avgABV' : 'mean',
        'medianABV' : 'median',
        'maxABV' : 'max',
        'minABV' : 'min',
        'ABVrange' : lambda x: max(x) - min(x)
    },
    'abv_listed' : {
        'num_abv_listed' : lambda x: int(sum(x)) 
    },
    'brewerID' : {
        'num_brewers' : lambda x: x.nunique()
    },
    'beer_style' : {
        'numBeerStyles' : lambda x : x.nunique()
    },
    'review_appearance' : {
        'avgAppearanceScore' : 'mean',
        'medianAppearanceScore' : 'median',
        'maxAppearanceScore' : 'max',
        'minAppearanceScore' : 'min',
        'appearanceScoreRange' : lambda x: max(x) - min(x)
    },
    'review_aroma' : {
        'avgAromaScore' : 'mean',
        'medianAromaScore' : 'median',
        'maxAromaScore' : 'max',
        'minAromaScore' : 'min',
        'aromaScoreRange' : lambda x: max(x) - min(x)
    },
    'review_palate' : {
        'avgPalateScore' : 'mean',
        'medianPalateScore' : 'median',
        'maxPalateScore' : 'max',
        'minPalateScore' : 'min',
        'palateScoreRange' : lambda x: max(x) - min(x)
    },
    'review_taste' : {
        'avgTasteScore' : 'mean',
        'medianTasteScore' : 'median',
        'maxTasteScore' : 'max',
        'minTasteScore' : 'min',
        'tasteScoreRange' : lambda x: max(x) - min(x)
    },
    'review_overall' : {
        'avgOverallScore' : 'mean',
        'medianOverallScore' : 'median',
        'maxOverallScore' : 'max',
        'minOverallScore' : 'min',
        'overallScoreRange' : lambda x: max(x) - min(x)
    },
    'text_lookup' : {
        'text_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'time_from_beginning' : {
        'avgTimestamp' : 'mean',
        'medianTimestamp' : 'median',
        'maxTimestamp' : 'max',
        'minTimestamp' : 'min',
        'TimestampRange' : lambda x: max(x) - min(x)
    },
    'weekday_central' : {
        'most_common_weekday' : lambda x: x.value_counts().index[0],
        'number_of_weekdays' : lambda x: x.nunique()
    },
    'weekend_review' : {
        'num_weekend_reviews' : 'sum'
    },
    'month' : {
        'most_common_month' : lambda x: x.value_counts().index[0],
        'number_of_months' : lambda x: x.nunique()
    },
    'distance_from_july' : {
        'avgMonthsFromJuly' : 'mean',
        'medianMonthsFromJuly' : 'median',
        'maxMonthsFromJuly' : 'max',
        'minMonthsFromJuly' : 'min',
        'monthsFromJulyRange' : lambda x: max(x) - min(x)
    },
    'vader_neg' : {
        'avgVaderNeg' : 'mean',
        'medianVaderNeg' : 'median',
        'maxVaderNeg' : 'max',
        'minVaderNeg' : 'min',
        'vaderNegRange' : lambda x: max(x) - min(x)
    },
    'vader_neu' : {
        'avgVaderNeu' : 'mean',
        'medianVaderNeu' : 'median',
        'maxVaderNeu' : 'max',
        'minVaderNeu' : 'min',
        'vaderNeuRange' : lambda x: max(x) - min(x)
    },
    'vader_pos' : {
        'avgVaderPos' : 'mean',
        'medianVaderPos' : 'median',
        'maxVaderPos' : 'max',
        'minVaderPos' : 'min',
        'vaderPosRange' : lambda x: max(x) - min(x)
    }
}
aggByUser_df = df.groupby('reviewer_username').agg(aggregation)"""

"aggregation={\n    'beerID' : {\n        'beerID_count' : 'count'\n    },\n    'abv' : {\n        'avgABV' : 'mean',\n        'medianABV' : 'median',\n        'maxABV' : 'max',\n        'minABV' : 'min',\n        'ABVrange' : lambda x: max(x) - min(x)\n    },\n    'abv_listed' : {\n        'num_abv_listed' : lambda x: int(sum(x)) \n    },\n    'brewerID' : {\n        'num_brewers' : lambda x: x.nunique()\n    },\n    'beer_style' : {\n        'numBeerStyles' : lambda x : x.nunique()\n    },\n    'review_appearance' : {\n        'avgAppearanceScore' : 'mean',\n        'medianAppearanceScore' : 'median',\n        'maxAppearanceScore' : 'max',\n        'minAppearanceScore' : 'min',\n        'appearanceScoreRange' : lambda x: max(x) - min(x)\n    },\n    'review_aroma' : {\n        'avgAromaScore' : 'mean',\n        'medianAromaScore' : 'median',\n        'maxAromaScore' : 'max',\n        'minAromaScore' : 'min',\n        'aromaScoreRange' : lambda x: max(x) - min(x)\n    },\n    'review_

In [13]:
# See above. This approach, of keeping aggregate features in lists was way too computationally expensive
"""aggregation={
    'beerID' : {
        'beerID_count' : 'count'
    },
    'abv_lookup' : {
        'abv_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'abv_listed' : {
        'num_abv_listed' : lambda x: int(sum(x)) 
    },
    'brewerID_lookup' : {
        'brewerID_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'beerStyle_lookup' : {
        'beerStyle_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'review_appearance_lookup' : {
        'reviewAppearance_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'review_aroma_lookup' : {
        'reviewAroma_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'review_palate_lookup' : {
        'reviewPalate_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'review_taste_lookup' : {
        'reviewTaste_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'review_overall_lookup' : {
        'reviewOverall_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'text_lookup' : {
        'text_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'time_from_beginning_lookup' : {
        'time_from_beginning_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'weekday_central_lookup' : {
        'weekday_central_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'weekend_review' : {
        'num_weekend_reviews' : 'sum'
    },
    'month_lookup' : {
        'month_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'distance_from_july_lookup' : {
        'distance_from_july_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'vader_neg_lookup' : {
        'vader_neg_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'vader_neu_lookup' : {
        'vader_neu_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'vader_pos_lookup' : {
        'vader_pos_list' : lambda x : [[item[0],item[2]] for item in x]
    }
}
aggByUser_df = df.groupby('reviewer_username').agg(aggregation)"""

"aggregation={\n    'beerID' : {\n        'beerID_count' : 'count'\n    },\n    'abv_lookup' : {\n        'abv_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'abv_listed' : {\n        'num_abv_listed' : lambda x: int(sum(x)) \n    },\n    'brewerID_lookup' : {\n        'brewerID_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'beerStyle_lookup' : {\n        'beerStyle_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'review_appearance_lookup' : {\n        'reviewAppearance_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'review_aroma_lookup' : {\n        'reviewAroma_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'review_palate_lookup' : {\n        'reviewPalate_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'review_taste_lookup' : {\n        'reviewTaste_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'review_overall_lookup' : {\n        'reviewOverall_list' 

In [14]:
#aggByUser_df['text_dict'] = aggByUser_df['text_lookup']['text_list'].apply(lambda x: {item[0]:item[1] for item in x})

#test=aggByUser_df['text_lookup']['text_dict'][4]
#{x[0]:x[1] for x in test}

In [15]:
#aggByUser_df.columns = aggByUser_df.columns.droplevel()

In [16]:
#aggByUser_df.head()

In [17]:
#aggByUser_df['weekend_review']

In [18]:
#aggByUser_df['beerID_count'].head()

In [19]:
#aggByUser_df['beerID_count'].mean()

In [20]:
"""aggregation2={
    'reviewer_username' : {
        'user_count' : 'count'
    },
    'abv' : {
        'ABV' : lambda x: x.value_counts().index[0]
    },
    'abv_listed' : {
        'num_abv_listed' : lambda x: x.value_counts().index[0]
    },
    'brewerID' : {
        'num_brewers' : lambda x: x.value_counts().index[0]
    },
    'beer_style' : {
        'numBeerStyles' : lambda x: x.value_counts().index[0]
    },
    'review_appearance_lookup' : {
        'reviewAppearance_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'review_aroma_lookup' : {
        'reviewAroma_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'review_palate_lookup' : {
        'reviewPalate_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'review_taste_lookup' : {
        'reviewTaste_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'review_overall_lookup' : {
        'reviewOverall_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'text_lookup' : {
        'text_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'time_from_beginning_lookup' : {
        'time_from_beginning_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'weekday_central_lookup' : {
        'weekday_central_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'weekend_review' : {
        'num_weekend_reviews' : 'sum'
    },
    'month_lookup' : {
        'month_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'distance_from_july_lookup' : {
        'distance_from_july_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'vader_neg_lookup' : {
        'vader_neg_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'vader_neu_lookup' : {
        'vader_neu_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'vader_pos_lookup' : {
        'vader_pos_list' : lambda x : [[item[1],item[2]] for item in x]
    }
}
aggByProduct_df = df.groupby('beerID').agg(aggregation2)"""

"aggregation2={\n    'reviewer_username' : {\n        'user_count' : 'count'\n    },\n    'abv' : {\n        'ABV' : lambda x: x.value_counts().index[0]\n    },\n    'abv_listed' : {\n        'num_abv_listed' : lambda x: x.value_counts().index[0]\n    },\n    'brewerID' : {\n        'num_brewers' : lambda x: x.value_counts().index[0]\n    },\n    'beer_style' : {\n        'numBeerStyles' : lambda x: x.value_counts().index[0]\n    },\n    'review_appearance_lookup' : {\n        'reviewAppearance_list' : lambda x : [[item[1],item[2]] for item in x]\n    },\n    'review_aroma_lookup' : {\n        'reviewAroma_list' : lambda x : [[item[1],item[2]] for item in x]\n    },\n    'review_palate_lookup' : {\n        'reviewPalate_list' : lambda x : [[item[1],item[2]] for item in x]\n    },\n    'review_taste_lookup' : {\n        'reviewTaste_list' : lambda x : [[item[1],item[2]] for item in x]\n    },\n    'review_overall_lookup' : {\n        'reviewOverall_list' : lambda x : [[item[1],item[2]] 

In [21]:
# Thia will now have to waint until aggretation from all the list features is done
# rate of reviewing - beerID_count/timeStampRange
#aggByProduct_df['rate_of_posting'] = aggByProduct_df['reviewer_username']['user_count']/aggByProduct_df['time_from_beginning']['TimestampRange']

# weekday variety index - number_of_weekdays/beerID_count
#aggByProduct_df['weekday_variety_index'] = aggByProduct_df['weekday_central']['number_of_weekdays']/aggByProduct_df['reviewer_username']['user_count']

# month variety index - number_of_months/beerID_count
#aggByProduct_df['month_variety_index'] = aggByProduct_df['month']['number_of_months']/aggByProduct_df['reviewer_username']['user_count']

# weekend review ratio - num_weekend_reviews/beerID_count
#aggByProduct_df['weekend_review_ratio'] = aggByProduct_df['weekend_review']['num_weekend_reviews']/aggByProduct_df['reviewer_username']['user_count']



In [22]:
#aggByProduct_df.columns = aggByProduct_df.columns.droplevel()

In [23]:
#aggByProduct_df.head()

In [24]:
#aggByProduct_df['reviewer_username'].mean()

## Collaborative filtering features


In [32]:
df.columns

Index(['beerID', 'beer_name', 'abv', 'abv_listed', 'brewerID', 'beer_style',
       'reviewer_username', 'review_appearance', 'review_aroma',
       'review_palate', 'review_taste', 'review_overall', 'review_text',
       'review_unix_time', 'weekday_central', 'day_of_year', 'month',
       'distance_from_july', 'text_filtered', 'text_simple', 'sentiment_vader',
       'text_lookup', 'abv_lookup', 'brewerID_lookup', 'beerStyle_lookup',
       'review_appearance_lookup', 'review_aroma_lookup',
       'review_palate_lookup', 'review_taste_lookup', 'review_overall_lookup',
       'vader_neg', 'vader_neu', 'vader_pos', 'time_from_beginning',
       'weekend_review', 'time_from_beginning_lookup',
       'weekday_central_lookup', 'month_lookup', 'distance_from_july_lookup',
       'vader_neg_lookup', 'vader_neu_lookup', 'vader_pos_lookup'],
      dtype='object')

In [33]:
df[['review_appearance', 'review_aroma', 'review_palate', 'review_taste', 'review_overall']]

,review_appearance,review_aroma,review_palate,review_taste,review_overall
2291588,5,8,4,10,20
1651460,5,6,5,9,17
2146614,3,10,4,9,18
2182000,4,7,5,9,17
2171356,3,7,2,6,10
2696165,4,10,4,9,18
1710138,4,6,2,2,3
144972,2,1,2,2,2
2694336,4,9,5,8,17
446495,2,3,1,3,5


We must split the training set in two. Half will be used to do collaborative filtering. The other half will be used to train a classifier. This 50-50 ratio picked slightly naively. May change this in the future.

In [34]:
#SVD_set, train_set = train_test_split(df, test_size=0.5)

In [26]:
#SVD_set.to_pickle(path='SVD_set.pkl')
#train_set.to_pickle(path='train_set.pkl')

SVD_set = pd.read_pickle('SVD_set.pkl')
train_set = pd.read_pickle('train_set.pkl')
#NB: RERUN ALL THE COLLABORATIVE FILTERING MODELS!!!

In [27]:
SVD_set.count()

beerID                        1315873
beer_name                     1315873
abv                           1315873
abv_listed                    1315873
brewerID                      1315873
beer_style                    1315873
reviewer_username             1315873
review_appearance             1315873
review_aroma                  1315873
review_palate                 1315873
review_taste                  1315873
review_overall                1315873
review_text                   1315873
review_unix_time              1315873
weekday_central               1315873
day_of_year                   1315873
month                         1315873
distance_from_july            1315873
text_filtered                 1315873
text_simple                   1315873
sentiment_vader               1315873
text_lookup                   1315873
abv_lookup                    1315873
brewerID_lookup               1315873
beerStyle_lookup              1315873
review_appearance_lookup      1315873
review_aroma

In [28]:
train_set.count()

beerID                        1315873
beer_name                     1315873
abv                           1315873
abv_listed                    1315873
brewerID                      1315873
beer_style                    1315873
reviewer_username             1315873
review_appearance             1315873
review_aroma                  1315873
review_palate                 1315873
review_taste                  1315873
review_overall                1315873
review_text                   1315873
review_unix_time              1315873
weekday_central               1315873
day_of_year                   1315873
month                         1315873
distance_from_july            1315873
text_filtered                 1315873
text_simple                   1315873
sentiment_vader               1315873
text_lookup                   1315873
abv_lookup                    1315873
brewerID_lookup               1315873
beerStyle_lookup              1315873
review_appearance_lookup      1315873
review_aroma

In [3]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(SVD_set[['reviewer_username', 'beerID', 'review_appearance']], reader)
data.split(n_folds=5)

appearanaceSVD = SVD()
evaluate(appearanaceSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/appearanaceSVD_dump')
dump.dump(file_name, algo=appearanaceSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1


KeyboardInterrupt: 

In [81]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(SVD_set[['reviewer_username', 'beerID', 'review_aroma']], reader)
data.split(n_folds=5)

aromaSVD = SVD()
evaluate(aromaSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/aromaSVD_dump')
dump.dump(file_name, algo=aromaSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 1.1902
MAE:  0.8873
------------
Fold 2
RMSE: 1.1907
MAE:  0.8882
------------
Fold 3
RMSE: 1.1888
MAE:  0.8855
------------
Fold 4
RMSE: 1.1868
MAE:  0.8849
------------
Fold 5
RMSE: 1.1899
MAE:  0.8883
------------
------------
Mean RMSE: 1.1893
Mean MAE : 0.8868
------------
------------


In [79]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(SVD_set[['reviewer_username', 'beerID', 'review_palate']], reader)
data.split(n_folds=5)

palateSVD = SVD()
evaluate(palateSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/palateSVD_dump')
dump.dump(file_name, algo=palateSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.6731
MAE:  0.5179
------------
Fold 2
RMSE: 0.6732
MAE:  0.5178
------------
Fold 3
RMSE: 0.6737
MAE:  0.5175
------------
Fold 4
RMSE: 0.6738
MAE:  0.5185
------------
Fold 5
RMSE: 0.6744
MAE:  0.5189
------------
------------
Mean RMSE: 0.6736
Mean MAE : 0.5181
------------
------------


In [82]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(SVD_set[['reviewer_username', 'beerID', 'review_taste']], reader)
data.split(n_folds=5)

tasteSVD = SVD()
evaluate(tasteSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/tasteSVD_dump')
dump.dump(file_name, algo=tasteSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 1.2252
MAE:  0.9163
------------
Fold 2
RMSE: 1.2243
MAE:  0.9148
------------
Fold 3
RMSE: 1.2239
MAE:  0.9150
------------
Fold 4
RMSE: 1.2256
MAE:  0.9160
------------
Fold 5
RMSE: 1.2290
MAE:  0.9194
------------
------------
Mean RMSE: 1.2256
Mean MAE : 0.9163
------------
------------


In [83]:
reader = Reader(rating_scale=(1, 20))
data = Dataset.load_from_df(SVD_set[['reviewer_username', 'beerID', 'review_overall']], reader)
data.split(n_folds=5)

overallSVD = SVD()
evaluate(overallSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/overallSVD_dump')
dump.dump(file_name, algo=overallSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 2.4827
MAE:  1.8149
------------
Fold 2
RMSE: 2.4896
MAE:  1.8182
------------
Fold 3
RMSE: 2.4831
MAE:  1.8161
------------
Fold 4
RMSE: 2.4791
MAE:  1.8122
------------
Fold 5
RMSE: 2.4970
MAE:  1.8230
------------
------------
Mean RMSE: 2.4863
Mean MAE : 1.8169
------------
------------


In [29]:
aggregation={
    'beerID' : {
        'beerID_count' : 'count'
    },
    'abv' : {
        'avgABV' : 'mean',
        'medianABV' : 'median',
        'maxABV' : 'max',
        'minABV' : 'min',
        'ABVrange' : lambda x: max(x) - min(x)
    },
    'abv_listed' : {
        'num_abv_listed' : lambda x: int(sum(x)) 
    },
    'brewerID' : {
        'num_brewers' : lambda x: x.nunique()
    },
    'beer_style' : {
        'numBeerStyles' : lambda x : x.nunique()
    },
    'review_appearance' : {
        'avgAppearanceScore' : 'mean',
        'medianAppearanceScore' : 'median',
        'maxAppearanceScore' : 'max',
        'minAppearanceScore' : 'min',
        'appearanceScoreRange' : lambda x: max(x) - min(x)
    },
    'review_aroma' : {
        'avgAromaScore' : 'mean',
        'medianAromaScore' : 'median',
        'maxAromaScore' : 'max',
        'minAromaScore' : 'min',
        'aromaScoreRange' : lambda x: max(x) - min(x)
    },
    'review_palate' : {
        'avgPalateScore' : 'mean',
        'medianPalateScore' : 'median',
        'maxPalateScore' : 'max',
        'minPalateScore' : 'min',
        'palateScoreRange' : lambda x: max(x) - min(x)
    },
    'review_taste' : {
        'avgTasteScore' : 'mean',
        'medianTasteScore' : 'median',
        'maxTasteScore' : 'max',
        'minTasteScore' : 'min',
        'tasteScoreRange' : lambda x: max(x) - min(x)
    },
    'review_overall' : {
        'avgOverallScore' : 'mean',
        'medianOverallScore' : 'median',
        'maxOverallScore' : 'max',
        'minOverallScore' : 'min',
        'overallScoreRange' : lambda x: max(x) - min(x)
    },
    'text_lookup' : {
        'text_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'time_from_beginning' : {
        'avgTimestamp' : 'mean',
        'medianTimestamp' : 'median',
        'maxTimestamp' : 'max',
        'minTimestamp' : 'min',
        'TimestampRange' : lambda x: max(x) - min(x)
    },
    'weekday_central' : {
        'most_common_weekday' : lambda x: x.value_counts().index[0],
        'number_of_weekdays' : lambda x: x.nunique()
    },
    'weekend_review' : {
        'num_weekend_reviews' : 'sum'
    },
    'month' : {
        'most_common_month' : lambda x: x.value_counts().index[0],
        'number_of_months' : lambda x: x.nunique()
    },
    'distance_from_july' : {
        'avgMonthsFromJuly' : 'mean',
        'medianMonthsFromJuly' : 'median',
        'maxMonthsFromJuly' : 'max',
        'minMonthsFromJuly' : 'min',
        'monthsFromJulyRange' : lambda x: max(x) - min(x)
    },
    'vader_neg' : {
        'avgVaderNeg' : 'mean',
        'medianVaderNeg' : 'median',
        'maxVaderNeg' : 'max',
        'minVaderNeg' : 'min',
        'vaderNegRange' : lambda x: max(x) - min(x)
    },
    'vader_neu' : {
        'avgVaderNeu' : 'mean',
        'medianVaderNeu' : 'median',
        'maxVaderNeu' : 'max',
        'minVaderNeu' : 'min',
        'vaderNeuRange' : lambda x: max(x) - min(x)
    },
    'vader_pos' : {
        'avgVaderPos' : 'mean',
        'medianVaderPos' : 'median',
        'maxVaderPos' : 'max',
        'minVaderPos' : 'min',
        'vaderPosRange' : lambda x: max(x) - min(x)
    }
}
aggByUser_df = SVD_set.groupby('reviewer_username').agg(aggregation)

/Users/conhealy/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [30]:
aggregation={
    'reviewer_username' : {
        'user_count' : 'count'
    },
    'abv' : {
        'avgABV' : 'mean',
        'medianABV' : 'median',
        'maxABV' : 'max',
        'minABV' : 'min',
        'ABVrange' : lambda x: max(x) - min(x)
    },
    'abv_listed' : {
        'num_abv_listed' : lambda x: int(sum(x)) 
    },
    'beer_style' : {
        'numBeerStyles' : lambda x: x.value_counts().index[0]
    },
    'review_appearance' : {
        'avgAppearanceScore' : 'mean',
        'medianAppearanceScore' : 'median',
        'maxAppearanceScore' : 'max',
        'minAppearanceScore' : 'min',
        'appearanceScoreRange' : lambda x: max(x) - min(x)
    },
    'review_aroma' : {
        'avgAromaScore' : 'mean',
        'medianAromaScore' : 'median',
        'maxAromaScore' : 'max',
        'minAromaScore' : 'min',
        'aromaScoreRange' : lambda x: max(x) - min(x)
    },
    'review_palate' : {
        'avgPalateScore' : 'mean',
        'medianPalateScore' : 'median',
        'maxPalateScore' : 'max',
        'minPalateScore' : 'min',
        'palateScoreRange' : lambda x: max(x) - min(x)
    },
    'review_taste' : {
        'avgTasteScore' : 'mean',
        'medianTasteScore' : 'median',
        'maxTasteScore' : 'max',
        'minTasteScore' : 'min',
        'tasteScoreRange' : lambda x: max(x) - min(x)
    },
    'review_overall' : {
        'avgOverallScore' : 'mean',
        'medianOverallScore' : 'median',
        'maxOverallScore' : 'max',
        'minOverallScore' : 'min',
        'overallScoreRange' : lambda x: max(x) - min(x)
    },
    'text_lookup' : {
        'text_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'time_from_beginning' : {
        'avgTimestamp' : 'mean',
        'medianTimestamp' : 'median',
        'maxTimestamp' : 'max',
        'minTimestamp' : 'min',
        'TimestampRange' : lambda x: max(x) - min(x)
    },
    'weekday_central' : {
        'most_common_weekday' : lambda x: x.value_counts().index[0],
        'number_of_weekdays' : lambda x: x.nunique()
    },
    'weekend_review' : {
        'num_weekend_reviews' : 'sum'
    },
    'month' : {
        'most_common_month' : lambda x: x.value_counts().index[0],
        'number_of_months' : lambda x: x.nunique()
    },
    'distance_from_july' : {
        'avgMonthsFromJuly' : 'mean',
        'medianMonthsFromJuly' : 'median',
        'maxMonthsFromJuly' : 'max',
        'minMonthsFromJuly' : 'min',
        'monthsFromJulyRange' : lambda x: max(x) - min(x)
    },
    'vader_neg' : {
        'avgVaderNeg' : 'mean',
        'medianVaderNeg' : 'median',
        'maxVaderNeg' : 'max',
        'minVaderNeg' : 'min',
        'vaderNegRange' : lambda x: max(x) - min(x)
    },
    'vader_neu' : {
        'avgVaderNeu' : 'mean',
        'medianVaderNeu' : 'median',
        'maxVaderNeu' : 'max',
        'minVaderNeu' : 'min',
        'vaderNeuRange' : lambda x: max(x) - min(x)
    },
    'vader_pos' : {
        'avgVaderPos' : 'mean',
        'medianVaderPos' : 'median',
        'maxVaderPos' : 'max',
        'minVaderPos' : 'min',
        'vaderPosRange' : lambda x: max(x) - min(x)
    }
}
aggByProduct_df = SVD_set.groupby('beerID').agg(aggregation)

/Users/conhealy/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [31]:
aggByUser_df.head()

time_from_beginning                            \
                       TimestampRange minTimestamp maxTimestamp   
reviewer_username                                                 
 -BB-99                             0     49593600     49593600   
 007lund                     22723200     71971200     94694400   
 0087341                            0    345340800    345340800   
 00cobraR                   176860800    130032000    306892800   
 0110x011                     3715200    269395200    273110400   

                                                    review_appearance  \
                  medianTimestamp  avgTimestamp medianAppearanceScore   
reviewer_username                                                       
 -BB-99                  49593600  4.959360e+07                   5.0   
 007lund                 74995200  7.814053e+07                   3.0   
 0087341                345340800  3.453408e+08                   5.0   
 00cobraR               208396800  2.052347e+08                   3.0   
 0110x011               269395200  2.702208e+08                   4.0   

                                                                              \
                  maxAppearanceScore appearanceScoreRange avgAppearanceScore   
reviewer_username                                                              
 -BB-99                            5                    0           5.000000   
 007lund                           5                    4           3.340426   
 0087341                           5                    0           5.000000   
 00cobraR                          5                    4           3.218284   
 0110x011                          5                    2           4.000000   

                                           ...         vader_neg  \
                  minAppearanceScore       ...       avgVaderNeg   
reviewer_username                          ...                     
 -BB-99                            5       ...          0.000000   
 007lund                           1       ...          0.067766   
 0087341                           5       ...          0.000000   
 00cobraR                          1       ...          0.051487   
 0110x011                          3       ...          0.033333   

                             month                        weekend_review  \
                  number_of_months most_common_month num_weekend_reviews   
reviewer_username                                                          
 -BB-99                          1                11                 0.0   
 007lund                        10                07                16.0   
 0087341                         1                03                 0.0   
 00cobraR                       12                05               273.0   
 0110x011                        2                10                 9.0   

                     brewerID    review_overall                  \
                  num_brewers overallScoreRange avgOverallScore   
reviewer_username                                                 
 -BB-99                     1                 0        8.000000   
 007lund                   32                17       11.893617   
 0087341                    1                 0       19.000000   
 00cobraR                 201                17       12.240672   
 0110x011                   9                 6       15.555556   

                                                                      
                  maxOverallScore medianOverallScore minOverallScore  
reviewer_username                                                     
 -BB-99                         8                8.0               8  
 007lund                       18               13.0               1  
 0087341                       19               19.0              19  
 00cobraR                      18               13.0               1  
 0110x011                      18               15.0              1

In [32]:
aggByUser_df.columns = aggByUser_df.columns.get_level_values(0)   

In [33]:
aggByUser_df.head()

,time_from_beginning,time_from_beginning,time_from_beginning,time_from_beginning,time_from_beginning,review_appearance,review_appearance,review_appearance,review_appearance,review_appearance,...,vader_neg,month,month,weekend_review,brewerID,review_overall,review_overall,review_overall,review_overall,review_overall
reviewer_username,,,,,,,,,,,,,,,,,,,,,
-BB-99,0,49593600,49593600,49593600,4.959360e+07,5.0,5,0,5.000000,5,...,0.000000,1,11,0.0,1,0,8.000000,8,8.0,8
007lund,22723200,71971200,94694400,74995200,7.814053e+07,3.0,5,4,3.340426,1,...,0.067766,10,07,16.0,32,17,11.893617,18,13.0,1
0087341,0,345340800,345340800,345340800,3.453408e+08,5.0,5,0,5.000000,5,...,0.000000,1,03,0.0,1,0,19.000000,19,19.0,19
00cobraR,176860800,130032000,306892800,208396800,2.052347e+08,3.0,5,4,3.218284,1,...,0.051487,12,05,273.0,201,17,12.240672,18,13.0,1
0110x011,3715200,269395200,273110400,269395200,2.702208e+08,4.0,5,2,4.000000,3,...,0.033333,2,10,9.0,9,6,15.555556,18,15.0,12


In [34]:
aggByProduct_df.columns = aggByProduct_df.columns.get_level_values(0)

In [35]:
aggByProduct_df.head()

,time_from_beginning,time_from_beginning,time_from_beginning,time_from_beginning,time_from_beginning,review_appearance,review_appearance,review_appearance,review_appearance,review_appearance,...,vader_neu,vader_neu,vader_neu,vader_neu,beer_style,review_overall,review_overall,review_overall,review_overall,review_overall
beerID,,,,,,,,,,,,,,,,,,,,,
1,339638400,2073600,341712000,156168000,1.673195e+08,4.0,5,4,4.151709,1,...,0.000,0.7690,1.000,0.752746,Dry Stout,19,13.668803,20,14.0,1
2,340070400,9072000,349142400,222220800,2.111082e+08,3.0,5,4,3.086154,1,...,0.000,0.8110,1.000,0.793006,Vienna,17,10.673846,20,11.0,3
3,332294400,17625600,349920000,286718400,2.718129e+08,3.0,5,4,3.151163,1,...,0.491,0.7930,0.509,0.784314,Heller Bock,17,11.691860,19,12.0,2
4,316915200,33955200,350870400,224812800,2.198255e+08,3.0,5,4,3.130081,1,...,0.461,0.7970,0.539,0.786114,Heller Bock,14,11.609756,17,12.0,3
5,329184000,19353600,348537600,247449600,2.450835e+08,3.0,5,4,3.224138,1,...,0.000,0.8065,1.000,0.796741,Amber Ale,13,11.114943,18,11.0,5


## Join dataframes together to build final training set

In [36]:
train_set['beerID'].count()

1315873

In [37]:
aggByUser_df['reviewer_username']=aggByUser_df.index

In [38]:
aggByProduct_df['beerID']=aggByProduct_df.index

In [39]:
aggByUser_df.head()

,time_from_beginning,time_from_beginning,time_from_beginning,time_from_beginning,time_from_beginning,review_appearance,review_appearance,review_appearance,review_appearance,review_appearance,...,month,month,weekend_review,brewerID,review_overall,review_overall,review_overall,review_overall,review_overall,reviewer_username
reviewer_username,,,,,,,,,,,,,,,,,,,,,
-BB-99,0,49593600,49593600,49593600,4.959360e+07,5.0,5,0,5.000000,5,...,1,11,0.0,1,0,8.000000,8,8.0,8,-BB-99
007lund,22723200,71971200,94694400,74995200,7.814053e+07,3.0,5,4,3.340426,1,...,10,07,16.0,32,17,11.893617,18,13.0,1,007lund
0087341,0,345340800,345340800,345340800,3.453408e+08,5.0,5,0,5.000000,5,...,1,03,0.0,1,0,19.000000,19,19.0,19,0087341
00cobraR,176860800,130032000,306892800,208396800,2.052347e+08,3.0,5,4,3.218284,1,...,12,05,273.0,201,17,12.240672,18,13.0,1,00cobraR
0110x011,3715200,269395200,273110400,269395200,2.702208e+08,4.0,5,2,4.000000,3,...,2,10,9.0,9,6,15.555556,18,15.0,12,0110x011


In [40]:
df_partMerge = pd.merge(train_set, aggByUser_df, on='reviewer_username', how='outer', 
                        suffixes=('_orig', '_userAgg'))

In [41]:
df_partMerge['brewerID_orig'].count()

1315873

In [42]:
df_partMerge.columns.values

array(['beerID_orig', 'beer_name', 'abv_orig', 'abv_listed_orig',
       'brewerID_orig', 'beer_style_orig', 'reviewer_username',
       'review_appearance_orig', 'review_aroma_orig', 'review_palate_orig',
       'review_taste_orig', 'review_overall_orig', 'review_text',
       'review_unix_time', 'weekday_central_orig', 'day_of_year',
       'month_orig', 'distance_from_july_orig', 'text_filtered',
       'text_simple', 'sentiment_vader', 'text_lookup_orig', 'abv_lookup',
       'brewerID_lookup', 'beerStyle_lookup', 'review_appearance_lookup',
       'review_aroma_lookup', 'review_palate_lookup',
       'review_taste_lookup', 'review_overall_lookup', 'vader_neg_orig',
       'vader_neu_orig', 'vader_pos_orig', 'time_from_beginning_orig',
       'weekend_review_orig', 'time_from_beginning_lookup',
       'weekday_central_lookup', 'month_lookup',
       'distance_from_july_lookup', 'vader_neg_lookup', 'vader_neu_lookup',
       'vader_pos_lookup', 'time_from_beginning_userAgg',
       

In [43]:
df_merged = pd.merge(df_partMerge, aggByProduct_df, left_on='beerID_orig', right_on='beerID', how='outer'
         , suffixes=('', '_prodAgg'))

In [65]:
# number of rows increases because some beers aren't reviewed at all
df_merged['beerID'].count()

1307955

In [51]:
#dfm1, dfm2 = np.array_split(df_merged, 2)
#dfm1.to_pickle(path='./df_merged1.pkl')
#dfm2.to_pickle(path='./df_merged2.pkl')

dfm1 = pd.read_pickle('df_merged1.pkl')
dfm2 = pd.read_pickle('df_merged2.pkl')
df_merged=pd.concat([dfm1,dfm2])



In [43]:
df_merged.columns.values


array(['beerID', 'beer_name', 'abv', 'abv_listed', 'brewerID',
       'beer_style', 'reviewer_username', 'review_appearance',
       'review_aroma', 'review_palate', 'review_taste', 'review_overall',
       'review_text', 'review_unix_time', 'weekday_central', 'day_of_year',
       'month', 'distance_from_july', 'text_filtered', 'text_simple',
       'sentiment_vader', 'text_lookup', 'abv_lookup', 'brewerID_lookup',
       'beerStyle_lookup', 'review_appearance_lookup',
       'review_aroma_lookup', 'review_palate_lookup',
       'review_taste_lookup', 'review_overall_lookup', 'vader_neg',
       'vader_neu', 'vader_pos', 'time_from_beginning', 'weekend_review',
       'time_from_beginning_lookup', 'weekday_central_lookup',
       'month_lookup', 'distance_from_july_lookup', 'vader_neg_lookup',
       'vader_neu_lookup', 'vader_pos_lookup', 'num_weekend_reviews',
       'brewerID_list', 'beerID_count', 'vader_neg_list',
       'distance_from_july_list', 'reviewAroma_list', 'text_list',

In [44]:
df_merged.head()

,beerID,beer_name,abv,abv_listed,brewerID,beer_style,reviewer_username,review_appearance,review_aroma,review_palate,...,vader_pos_list_prodAgg,month_list_prodAgg,reviewOverall_list_prodAgg,reviewAppearance_list_prodAgg,reviewTaste_list_prodAgg,vader_neu_list_prodAgg,reviewPalate_list_prodAgg,num_abv_listed_prodAgg,time_from_beginning_list_prodAgg,numBeerStyles
0,5868,St Peters Organic Ale,4.5,True,974,Bitter,abemorsten,3,6,4,...,"[[ abemorsten, 0.179], [ hayduke, 0.068], [ co...","[[ abemorsten, 10], [ hayduke, 01], [ colts901...","[[ abemorsten, 14], [ hayduke, 14], [ colts901...","[[ abemorsten, 3], [ hayduke, 3], [ colts9016,...","[[ abemorsten, 6], [ hayduke, 6], [ colts9016,...","[[ abemorsten, 0.757], [ hayduke, 0.913], [ co...","[[ abemorsten, 4], [ hayduke, 3], [ colts9016,...",True,"[[ abemorsten, 300412800], [ hayduke, 21366720...",Bitter
1,5868,St Peters Organic Ale,4.5,True,974,Bitter,jazz88,3,6,3,...,"[[ abemorsten, 0.179], [ hayduke, 0.068], [ co...","[[ abemorsten, 10], [ hayduke, 01], [ colts901...","[[ abemorsten, 14], [ hayduke, 14], [ colts901...","[[ abemorsten, 3], [ hayduke, 3], [ colts9016,...","[[ abemorsten, 6], [ hayduke, 6], [ colts9016,...","[[ abemorsten, 0.757], [ hayduke, 0.913], [ co...","[[ abemorsten, 4], [ hayduke, 3], [ colts9016,...",True,"[[ abemorsten, 300412800], [ hayduke, 21366720...",Bitter
2,5868,St Peters Organic Ale,4.5,True,974,Bitter,zach8270,3,6,3,...,"[[ abemorsten, 0.179], [ hayduke, 0.068], [ co...","[[ abemorsten, 10], [ hayduke, 01], [ colts901...","[[ abemorsten, 14], [ hayduke, 14], [ colts901...","[[ abemorsten, 3], [ hayduke, 3], [ colts9016,...","[[ abemorsten, 6], [ hayduke, 6], [ colts9016,...","[[ abemorsten, 0.757], [ hayduke, 0.913], [ co...","[[ abemorsten, 4], [ hayduke, 3], [ colts9016,...",True,"[[ abemorsten, 300412800], [ hayduke, 21366720...",Bitter
3,5868,St Peters Organic Ale,4.5,True,974,Bitter,Angeloregon,4,7,3,...,"[[ abemorsten, 0.179], [ hayduke, 0.068], [ co...","[[ abemorsten, 10], [ hayduke, 01], [ colts901...","[[ abemorsten, 14], [ hayduke, 14], [ colts901...","[[ abemorsten, 3], [ hayduke, 3], [ colts9016,...","[[ abemorsten, 6], [ hayduke, 6], [ colts9016,...","[[ abemorsten, 0.757], [ hayduke, 0.913], [ co...","[[ abemorsten, 4], [ hayduke, 3], [ colts9016,...",True,"[[ abemorsten, 300412800], [ hayduke, 21366720...",Bitter
4,5868,St Peters Organic Ale,4.5,True,974,Bitter,bierkoning,3,6,2,...,"[[ abemorsten, 0.179], [ hayduke, 0.068], [ co...","[[ abemorsten, 10], [ hayduke, 01], [ colts901...","[[ abemorsten, 14], [ hayduke, 14], [ colts901...","[[ abemorsten, 3], [ hayduke, 3], [ colts9016,...","[[ abemorsten, 6], [ hayduke, 6], [ colts9016,...","[[ abemorsten, 0.757], [ hayduke, 0.913], [ co...","[[ abemorsten, 4], [ hayduke, 3], [ colts9016,...",True,"[[ abemorsten, 300412800], [ hayduke, 21366720...",Bitter


### Remove the example in question from each lookup list
This way the specific rreview features are not embedded in the aggregate values. 

In [45]:
df_merged[['beerID','beerID_count']].head()

,beerID,beerID_count
0,5868,146
1,5868,1041
2,5868,1241
3,5868,949
4,5868,2424


In [46]:
def removekey(d, key):
    r = dict(d)
    del r[key]
    return r

In [47]:
df_merged.index.max()

1315872

In [50]:
df_merged.iloc[[1315872]]

,beerID,beer_name,abv,abv_listed,brewerID,beer_style,reviewer_username,review_appearance,review_aroma,review_palate,...,vader_pos_list_prodAgg,month_list_prodAgg,reviewOverall_list_prodAgg,reviewAppearance_list_prodAgg,reviewTaste_list_prodAgg,vader_neu_list_prodAgg,reviewPalate_list_prodAgg,num_abv_listed_prodAgg,time_from_beginning_list_prodAgg,numBeerStyles
1315872,8647,Blackfoot River Woollybugger Wheat,5.9,True,1467,German Hefeweizen,buzzlightbeer,5,10,5,...,"[[ buzzlightbeer, 0.147]]","[[ buzzlightbeer, 10]]","[[ buzzlightbeer, 20]]","[[ buzzlightbeer, 5]]","[[ buzzlightbeer, 10]]","[[ buzzlightbeer, 0.853]]","[[ buzzlightbeer, 5]]",True,"[[ buzzlightbeer, 268358400]]",German Hefeweizen


In [2]:
df_merged['vader_neg_list2'] = df_merged.apply(lambda row: removekey(dict(row['vader_neg_list']), int(row['beerID'])), axis=1)

NameError: name 'df_merged' is not defined

In [ ]:
df_merged['vader_neg_list2'] = df.apply(lambda row: [item for item in row['vader_neg_list'] if item[0]!=row['beerID'] ])

2

In [ ]:
# We woould not have review information in cases where we're trying to 
# recommend something that's not been reviewed before so we can drop all that stuff
df_merged.drop(['review_appearance', 'review_aroma', 'review_palate', 'review_taste',
               'review_overall', 'review_text', 'review_unix_time', 'weekday_central',
               'day_of_year', 'month', 'distance_from_july', 'text_filtered', 
               'text_simple', 'sentiment_vader', 'vader_neg', 'vader_neu',
               'vader_pos', 'time_from_beginning'], axis=1)

In [ ]:
# brewer variety index - numBrewers/beerID_count
aggByUser_df['beer_variety_index'] = aggByUser_df['num_brewers']/aggByUser_df['beerID_count']

# beer style variety index - numBeerStyles/beerID_count
aggByUser_df['brewer_variety_index'] = aggByUser_df['numBeerStyles']/aggByUser_df['beerID_count']

# rate of reviewing - beerID_count/timeStampRange
aggByUser_df['rate_of_posting'] = aggByUser_df['beerID_count']/aggByUser_df['TimestampRange']

# weekday variety index - number_of_weekdays/beerID_count
aggByUser_df['weekday_variety_index'] = aggByUser_df['number_of_weekdays']/aggByUser_df['beerID_count']

# month variety index - number_of_months/beerID_count
aggByUser_df['month_variety_index'] = aggByUser_df['number_of_months']/aggByUser_df['beerID_count']

# weekend review ratio - num_weekend_reviews/beerID_count
aggByUser_df['weekend_review_ratio'] = aggByUser_df['num_weekend_reviews']/aggByUser_df['beerID_count']


